# 📦 Optimizing Logistics with Data: Delhivery Case Study
Delhivery is one of India's leading logistics and supply chain companies. Known for its extensive reach and advanced delivery solutions, the company integrates cutting-edge technology to enable timely and reliable services across diverse regions.
# 🔍 Project Objective
This project focuses on analyzing logistics and delivery performance using Delhivery’s raw operational data. The goal is to extract meaningful insights that can improve logistics strategies, route optimization, and delivery efficiency.

In [66]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [67]:
delivery_data = pd.read_csv('delhivery.csv')

In [68]:
delivery_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144867 entries, 0 to 144866
Data columns (total 24 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   data                            144867 non-null  object 
 1   trip_creation_time              144867 non-null  object 
 2   route_schedule_uuid             144867 non-null  object 
 3   route_type                      144867 non-null  object 
 4   trip_uuid                       144867 non-null  object 
 5   source_center                   144867 non-null  object 
 6   source_name                     144574 non-null  object 
 7   destination_center              144867 non-null  object 
 8   destination_name                144606 non-null  object 
 9   od_start_time                   144867 non-null  object 
 10  od_end_time                     144867 non-null  object 
 11  start_scan_to_end_scan          144867 non-null  float64
 12  is_cutoff       

# 💡 Potential Use Cases
• Trip Efficiency Analysis: Evaluate the performance of different trips and identify delays or inefficiencies.

• Route Optimization: Understand how delivery routes are planned and executed using open-source routing engines.

• Transportation Modes: Assess how different types of transport affect delivery times.

• Performance Benchmarking: Track delivery KPIs such as time taken, distance covered, and trip success rates.


# 🧠 My Contribution
## 🧹 1. Data Cleaning & Feature Engineering
Processed and sanitized raw fields (e.g., timestamps, distances, geolocation).
Extracted features such as:
Average delivery time per route.
Delay metrics per trip or region.
Trip frequency by transport type.
## 📊 2. Exploratory Data Analysis (EDA)
Visualized trip durations, distances, and delays.
Identified outliers and anomalies.
Grouped data by city, zone, and transport type to find performance gaps.
## 📈 3. Insights & Recommendations
Suggested route optimization strategies based on trip repetition patterns.
Proposed filters for early detection of delivery failures.
Created dashboards for dynamic KPI monitoring.

In [69]:
# Check for missing values
print("Missing values in each column:")
print(delivery_data.isnull().sum())

Missing values in each column:
data                                0
trip_creation_time                  0
route_schedule_uuid                 0
route_type                          0
trip_uuid                           0
source_center                       0
source_name                       293
destination_center                  0
destination_name                  261
od_start_time                       0
od_end_time                         0
start_scan_to_end_scan              0
is_cutoff                           0
cutoff_factor                       0
cutoff_timestamp                    0
actual_distance_to_destination      0
actual_time                         0
osrm_time                           0
osrm_distance                       0
factor                              0
segment_actual_time                 0
segment_osrm_time                   0
segment_osrm_distance               0
segment_factor                      0
dtype: int64


### Handling Missing Values

During data inspection, I found missing values in the `source_name` and `destination_name` columns.  
To address this, I explored the possibility of imputing these values based on similarities with other columns such as:

- `source_lat` and `source_long`
- `destination_lat` and `destination_long`
- `trip_id` or `order_time`

This strategy helps retain valuable rows and ensures data completeness where patterns are identifiable.


In [70]:
# Asegurar que las columnas de coordenadas existan
for col in ['source_lat', 'source_long', 'destination_lat', 'destination_long']:
    if col not in delivery_data.columns:
        delivery_data[col] = np.nan

# --- Rellenar source_name basado en coordenadas ---
def fill_source_name(row):
    if pd.isna(row['source_name']):
        matches = delivery_data.loc[
            (delivery_data['source_lat'] == row['source_lat']) &
            (delivery_data['source_long'] == row['source_long']) &
            delivery_data['source_name'].notna(), 'source_name'
        ]
        if not matches.empty:
            return matches.mode().iloc[0]
    return row['source_name']

delivery_data['source_name'] = delivery_data.apply(fill_source_name, axis=1)

# --- Rellenar destination_name basado en coordenadas ---
def fill_destination_name(row):
    if pd.isna(row['destination_name']):
        matches = delivery_data.loc[
            (delivery_data['destination_lat'] == row['destination_lat']) &
            (delivery_data['destination_long'] == row['destination_long']) &
            delivery_data['destination_name'].notna(), 'destination_name'
        ]
        if not matches.empty:
            return matches.mode().iloc[0]
    return row['destination_name']

delivery_data['destination_name'] = delivery_data.apply(fill_destination_name, axis=1)

# Verificación final
print("✅ Missing source_name after fill:", delivery_data['source_name'].isna().sum())
print("✅ Missing destination_name after fill:", delivery_data['destination_name'].isna().sum())


✅ Missing source_name after fill: 293
✅ Missing destination_name after fill: 261


In [71]:
# Ver filas con source_name faltante
missing_source = delivery_data[delivery_data['source_name'].isna()]
# Ver si sus coordenadas existen más de una vez
source_coords_counts = delivery_data.groupby(['source_lat', 'source_long']).size().reset_index(name='count')
missing_source = missing_source.merge(source_coords_counts, on=['source_lat', 'source_long'], how='left')
print("Source coordinates that appear only once:")
print(missing_source[missing_source['count'] == 1].shape[0])

# Ver filas con destination_name faltante
missing_dest = delivery_data[delivery_data['destination_name'].isna()]
# Ver si sus coordenadas existen más de una vez
dest_coords_counts = delivery_data.groupby(['destination_lat', 'destination_long']).size().reset_index(name='count')
missing_dest = missing_dest.merge(dest_coords_counts, on=['destination_lat', 'destination_long'], how='left')
print("Destination coordinates that appear only once:")
print(missing_dest[missing_dest['count'] == 1].shape[0])


Source coordinates that appear only once:
0
Destination coordinates that appear only once:
0


In [72]:
def check_non_null_names(missing_df, lat_col, long_col, name_col):
    results = []
    grouped = missing_df.groupby([lat_col, long_col])
    for coords, group in grouped:
        lat, long = coords
        # Buscar si hay nombres no nulos en todas las filas con esa coordenada en el dataset original
        valid_names = delivery_data.loc[
            (delivery_data[lat_col] == lat) & 
            (delivery_data[long_col] == long) & 
            delivery_data[name_col].notna(),
            name_col
        ].unique()
        results.append((coords, valid_names))
    return results

# Revisar source_name
source_check = check_non_null_names(missing_source, 'source_lat', 'source_long', 'source_name')
print("Coordenadas con valores válidos para 'source_name':")
for coords, names in source_check:
    if len(names) == 0:
        print(f"{coords} -> No valid names")
    else:
        print(f"{coords} -> Valid names: {names}")

# Revisar destination_name
dest_check = check_non_null_names(missing_dest, 'destination_lat', 'destination_long', 'destination_name')
print("\nCoordenadas con valores válidos para 'destination_name':")
for coords, names in dest_check:
    if len(names) == 0:
        print(f"{coords} -> No valid names")
    else:
        print(f"{coords} -> Valid names: {names}")


Coordenadas con valores válidos para 'source_name':

Coordenadas con valores válidos para 'destination_name':


In [73]:
delivery_data['source_name'] = delivery_data['source_name'].fillna('Unknown Source')
delivery_data['destination_name'] = delivery_data['destination_name'].fillna('Unknown Destination')


# Handling Missing Values in `source_name` and `destination_name`

## Overview

The dataset contains missing values in the `source_name` and `destination_name` columns. To ensure data completeness, we performed imputation based on geographic coordinates.

## Steps Taken

1. **Imputation Based on Coordinates**

   - For rows with missing `source_name`, search other rows with the same `source_lat` and `source_long` that have a valid `source_name`.
   - For rows with missing `destination_name`, do the same using `destination_lat` and `destination_long`.
   - Use the most frequent (`mode`) name found to fill missing values.

2. **Verification**

   - After applying the imputation, count how many missing values remain in each column.

3. **Analysis of Remaining Missing Values**

   - Check if the coordinates corresponding to missing values are unique or appear multiple times.
   - Confirm that no valid names exist in the dataset for these coordinates, preventing further imputation.

4. **Final Fill**

   - Fill remaining missing values with placeholders: `"Unknown Source"` and `"Unknown Destination"`.

---

## Code Summary

```python
# Ensure coordinate columns exist
for col in ['source_lat', 'source_long', 'destination_lat', 'destination_long']:
    if col not in delivery_data.columns:
        delivery_data[col] = np.nan

# Functions to fill missing names based on coordinates
def fill_source_name(row):
    if pd.isna(row['source_name']):
        matches = delivery_data.loc[
            (delivery_data['source_lat'] == row['source_lat']) &
            (delivery_data['source_long'] == row['source_long']) &
            delivery_data['source_name'].notna(), 'source_name'
        ]
        if not matches.empty:
            return matches.mode().iloc[0]
    return row['source_name']

def fill_destination_name(row):
    if pd.isna(row['destination_name']):
        matches = delivery_data.loc[
            (delivery_data['destination_lat'] == row['destination_lat']) &
            (delivery_data['destination_long'] == row['destination_long']) &
            delivery_data['destination_name'].notna(), 'destination_name'
        ]
        if not matches.empty:
            return matches.mode().iloc[0]
    return row['destination_name']

# Apply the functions
delivery_data['source_name'] = delivery_data.apply(fill_source_name, axis=1)
delivery_data['destination_name'] = delivery_data.apply(fill_destination_name, axis=1)

# Check remaining missing values
print("Missing source_name after fill:", delivery_data['source_name'].isna().sum())
print("Missing destination_name after fill:", delivery_data['destination_name'].isna().sum())

# Fill remaining missing values with placeholders
delivery_data['source_name'] = delivery_data['source_name'].fillna('Unknown Source')
delivery_data['destination_name'] = delivery_data['destination_name'].fillna('Unknown Destination')


In [79]:
delivery_data['od_end_time'] = pd.to_datetime(delivery_data['od_end_time'], errors='coerce')
delivery_data['od_start_time'] = pd.to_datetime(delivery_data['od_start_time'], errors='coerce')
delivery_data['od_duration'] = (delivery_data['od_end_time'] - delivery_data['od_start_time']).dt.total_seconds() / 3600.0